In [5]:
## Langchain 1
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [6]:
# Langsmith tracking and tracing
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")




In [7]:
from langchain_openai import ChatOpenAI
llm= ChatOpenAI(
    model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x0000016D4A499A90> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000016D4A49A490> root_client=<openai.OpenAI object at 0x0000016D4A499F90> root_async_client=<openai.AsyncOpenAI object at 0x0000016D4A49A0D0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [8]:
# have paid version to use gpt-4o
# Example usage of the LLM
# result = llm.invoke("What is the capital of France?")
# print(result)

In [9]:
# free models 
from langchain_groq import ChatGroq
model= ChatGroq(model="gemma2-9b-it")
print(model)

client=<groq.resources.chat.completions.Completions object at 0x0000016D51E79A90> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016D51E7A7B0> model_name='gemma2-9b-it' model_kwargs={} groq_api_key=SecretStr('**********')


In [10]:
model.invoke("What is the capital of France?")  # Example usage of the Groq 

AIMessage(content='The capital of France is **Paris**. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 16, 'total_tokens': 28, 'completion_time': 0.021818182, 'prompt_time': 0.001828819, 'queue_time': 0.24300673, 'total_time': 0.023647001}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d9a8d5b5-2b9b-4bd9-9888-a28499f1a2fa-0', usage_metadata={'input_tokens': 16, 'output_tokens': 12, 'total_tokens': 28})

In [11]:
### Prompt Engineering

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{question}"),
])

prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [12]:
### Chaining LLMs
chain = prompt | model
chain

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000016D51E79A90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000016D51E7A7B0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
response = chain.invoke({"question": "Explain CNN in computer vision."})
print(response)

content='## CNNs for Computer Vision: Seeing the World Through Artificial Eyes\n\nImagine you\'re teaching a computer to recognize a cat in a picture. You wouldn\'t just show it the whole image at once, right? You\'d point out features like pointy ears, whiskers, and a fluffy tail, helping it understand what makes a cat, a cat.\n\nThat\'s essentially what Convolutional Neural Networks (CNNs) do! They\'re a special type of artificial neural network designed specifically for processing image data.\n\n**Here\'s a breakdown:**\n\n1. **Convolutional Layers:** Think of these as the "feature detectors." They scan the image with small filters (also called kernels) that highlight specific features like edges, corners, or textures. Each filter learns to recognize a particular pattern.\n\n2. **Pooling Layers:** These layers downsample the image, reducing its size while retaining important information. This helps the network become more robust to small changes in the image and makes processing fas

In [14]:
## Output Parser
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
# Example usage of the output parser
chain = prompt | model | output_parser
response = chain.invoke({"question": "Explain CNN in computer vision."})
print(response)

Imagine you're trying to teach a computer to recognize a cat in a picture. You could try to describe all the features of a cat – pointy ears, whiskers, fluffy tail – but that's hard to do precisely and computers aren't great at understanding complex descriptions.

That's where Convolutional Neural Networks (CNNs) come in! They're like specialized artificial brains designed specifically for understanding images. 

Here's how they work:

**1. Convolutional Layers:** These layers are the heart of a CNN. They use small filters (think of them as magnifying glasses) to scan the image. Each filter looks for specific patterns, like edges, corners, or textures.

**2. Pooling Layers:** After the convolutional layers, pooling layers shrink the image size while retaining the most important information. This helps make the network more efficient and robust to small changes in the image.

**3. Fully Connected Layers:** Finally, the processed information from the convolutional and pooling layers is f

In [15]:
### JSON Output Parser
from langchain_core.output_parsers import JsonOutputParser
json_output_parser = JsonOutputParser()
json_output_parser.get_format_instructions()


'Return a JSON object.'

In [16]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
json_output_parser = JsonOutputParser()

prompt=PromptTemplate(
    template="Answer the query \n {format_instructions}\n {question}",
    input_variables=["question"],
    partial_variables={"format_instructions": json_output_parser.get_format_instructions()},
)


In [17]:
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template='Answer the query \n {format_instructions}\n {question}')

In [18]:
chain=prompt | model | json_output_parser
response =chain.invoke({"question": "tell me about transformmers in NLP."});print(response)

{'name': 'Transformers in NLP', 'description': 'Transformer models are a revolutionary architecture in natural language processing (NLP) known for their ability to handle long-range dependencies in text effectively. They have achieved state-of-the-art results in a wide range of NLP tasks.', 'key_features': ['**Attention Mechanism:** Transformers rely heavily on the attention mechanism, which allows the model to focus on relevant parts of the input sequence when processing each word.', '**Encoder-Decoder Structure:** Most transformer models have an encoder-decoder structure, where the encoder processes the input sequence and the decoder generates the output sequence.', '**Parallel Processing:** Transformers can process input sequences in parallel, significantly speeding up training and inference compared to recurrent neural networks.', '**Positional Encodings:** Since transformers lack inherent sequential information, they use positional encodings to embed the order of words in the inpu

In [19]:
### OutputParser
from langchain_core.output_parsers import XMLOutputParser
output_parser=XMLOutputParser()
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert AI Engineer.<response><answer>Your answer here</answer></response>.Provide me answer based on the question"),
        ("user","{input}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer.<response><answer>Your answer here</answer></response>.Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [20]:
### OutputParser
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate

output_parser=XMLOutputParser()

prompt=PromptTemplate(
    template="Answer the user query \n {format_instruction}\n {query}\n ",
    input_variables=["query"],
    partial_variables={"format_instruction":output_parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'The output should be formatted as a XML file.\n1. Output should conform to the tags below.\n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema.\n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\nNone\n```'}, template='Answer the user query \n {format_instruction}\n {query}\n ')

In [21]:
chain=prompt|model
response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)

content='<response>\n  <name>Langsmith</name>\n  <description>Langsmith is an open-source platform for developing and deploying large language models (LLMs). It provides a framework for training, evaluating, and fine-tuning LLMs, as well as tools for deploying them as APIs.</description>\n  <features>\n    <feature>Modular and extensible architecture</feature>\n    <feature>Support for multiple LLM architectures</feature>\n    <feature>Comprehensive set of training and evaluation tools</feature>\n    <feature>Easy deployment as APIs</feature>\n  </features>\n  <creator>The Langsmith team</creator>\n</response> \n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 148, 'prompt_tokens': 195, 'total_tokens': 343, 'completion_time': 0.269090909, 'prompt_time': 0.008926806, 'queue_time': 0.24496708299999997, 'total_time': 0.278017715}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--cce48146-

In [22]:
##output parser
#from langchain_core.output_parsers import XMLOutputParser
from langchain.output_parsers.xml import XMLOutputParser

# XML Output Parser
output_parser = XMLOutputParser()

# Prompt that instructs the model to return XML
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond in this XML format: <response><answer>Your answer here</answer></response>"),
    ("human", "{input}")
])

# Build the chain
chain = prompt | model

# Run the chain
#response = chain.invoke({"input": "What is LangChain?"})

raw_output =chain.invoke({"input": "What is LangChain?"})

# Print result
print(raw_output)


content='<response><answer>LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It provides tools and components for tasks like prompting, memory management, chaining together different LLMs and other AI services, and integrating with external data sources.</answer></response>\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 39, 'total_tokens': 105, 'completion_time': 0.12, 'prompt_time': 0.002265799, 'queue_time': 0.24448298999999998, 'total_time': 0.122265799}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--9ef7525c-34cf-4648-a652-fc0a0e2e8613-0' usage_metadata={'input_tokens': 39, 'output_tokens': 66, 'total_tokens': 105}


In [23]:
## With Pydantic
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

from langchain_groq import ChatGroq
model= ChatGroq(model="gemma2-9b-it")



# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

In [24]:
### Without Pydantic
joke_query = "Tell me a joke ."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'joke': "Why don't scientists trust atoms? Because they make up everything!"}

In [25]:

from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate


actor_query = "Generate the shortened filmography for Tom Hanks."

output = model.invoke(
    f"""{actor_query}
Please enclose the movies in <movie></movie> tags"""
)

print(output.content)

Here is a shortened filmography for Tom Hanks, with movies enclosed in <movie></movie> tags:

* <movie>Splash</movie>
* <movie>Big</movie>
* <movie>Philadelphia</movie>
* <movie>Forrest Gump</movie>
* <movie>Saving Private Ryan</movie>
* <movie>Toy Story</movie>
* <movie>Cast Away</movie>
* <movie>The Da Vinci Code</movie>
* <movie>Apollo 13</movie> 


This list highlights some of his most popular and critically acclaimed roles. 




In [26]:
from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
model= ChatGroq(model="gemma2-9b-it")

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')